In [ ]:
# Install pyfuzzy-toolbox
!pip install pyfuzzy-toolbox -q

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import fuzzy_systems as fs
from fuzzy_systems import SugenoSystem

%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11
plt.rcParams['figure.dpi'] = 100

In [ ]:
# ============================================================================
# Create Sugeno Zero-Order (Singleton) System
# ============================================================================
system = SugenoSystem(order=0)

# ============================================================================
# INPUT: Student Grade (0 to 10)
# ============================================================================
system.add_input('grade', (0, 10))
system.add_term('grade', 'low', 'triangular', (0, 0, 5))
system.add_term('grade', 'medium', 'triangular', (0, 5, 10))
system.add_term('grade', 'high', 'triangular', (5, 10, 10))

# ============================================================================
# OUTPUT: Performance (constant values)
# ============================================================================
system.add_output('performance', (0, 10))

In [ ]:
x_vals = np.linspace(0, 10, 200)

plt.figure(figsize=(12, 6))

colors = {'low': 'b', 'medium': 'y', 'high': 'r'}
for term_name, term in system.input_variables['grade'].terms.items():
    mu = term.membership(x_vals)
    plt.plot(x_vals, mu, colors[term_name], label=term_name, linewidth=2)

plt.title('Membership Functions: Student Grade', fontsize=14, fontweight='bold')
plt.xlabel('Grade (0-10)', fontsize=12)
plt.ylabel('Membership Degree', fontsize=12)
plt.legend(loc='upper center', fontsize=11, ncol=3)
plt.grid(True, alpha=0.3)
plt.ylim([-0.05, 1.05])
plt.show()

In [ ]:
# ============================================================================
# Sugeno Zero-Order Rule Base
# ============================================================================
# Define rules with constant output values
system.add_rules([
    ('low', 2.0),      # IF grade is low THEN performance = 2.0
    ('medium', 6.0),   # IF grade is medium THEN performance = 6.0
    ('high', 9.0)      # IF grade is high THEN performance = 9.0
])

In [ ]:
# ============================================================================
# Detailed Step-by-Step Example
# ============================================================================
grade_example = 6.5

print("=" * 80)
print("SUGENO ZERO-ORDER INFERENCE EXAMPLE")
print("=" * 80)
print()
print(f"Input Grade: {grade_example}/10")
print()

# Fuzzification
print("Step 1: Fuzzification")
print("-" * 80)
for term_name, term in system.input_variables['grade'].terms.items():
    mu = term.membership(grade_example)
    if mu > 0.01:
        print(f"   μ({term_name}) = {mu:.3f}")

# Evaluate system
result = system.evaluate({'grade': grade_example})
performance = result['performance']

print()
print("Step 2: Rule Aggregation & Defuzzification")
print("-" * 80)
print(f"   Output Performance: {performance:.2f}")
print()
print("=" * 80)

In [ ]:
grade_example = 6.5
result = system.evaluate({'grade': grade_example})
performance = result['performance']

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# ============================================================================
# Left plot: Input membership
# ============================================================================
x_vals = np.linspace(0, 10, 200)
colors = {'low': 'b', 'medium': 'y', 'high': 'r'}
for term_name, term in system.input_variables['grade'].terms.items():
    mu = term.membership(x_vals)
    axes[0].plot(x_vals, mu, colors[term_name], label=term_name, linewidth=2)

axes[0].axvline(x=grade_example, color='red', linestyle='--', linewidth=2,
                label=f'Grade = {grade_example}')
axes[0].set_title(f'Fuzzification: Grade = {grade_example}', fontsize=12, fontweight='bold')
axes[0].set_xlabel('Grade (0-10)', fontsize=11)
axes[0].set_ylabel('Membership Degree', fontsize=11)
axes[0].legend(fontsize=10)
axes[0].grid(True, alpha=0.3)
axes[0].set_ylim([-0.05, 1.05])

# ============================================================================
# Right plot: Rule outputs
# ============================================================================
rules_info = [('low', 2.0), ('medium', 6.0), ('high', 9.0)]
output_values = [val for _, val in rules_info]
rule_names = [name for name, _ in rules_info]
activations = [system.input_variables['grade'].terms[name].membership(grade_example) 
               for name, _ in rules_info]

bars = axes[1].bar(rule_names, output_values, color=['b', 'y', 'r'], alpha=0.7, 
                   edgecolor='black', linewidth=2)
axes[1].axhline(y=performance, color='green', linestyle='--', linewidth=2,
                label=f'Final Output = {performance:.2f}')
axes[1].set_title('Rule Outputs & Defuzzified Result', fontsize=12, fontweight='bold')
axes[1].set_ylabel('Performance Value', fontsize=11)
axes[1].legend(fontsize=10)
axes[1].grid(True, alpha=0.3, axis='y')
axes[1].set_ylim([0, 10])

plt.tight_layout()
plt.show()

In [ ]:
print("=" * 80)
print("TESTING SUGENO ZERO-ORDER SYSTEM")
print("=" * 80)
print()

print(f"{'Grade':<10} {'Performance':<15} {'Category'}")
print("-" * 80)

test_grades = [2.0, 4.0, 5.0, 6.5, 7.5, 8.0, 9.5]
for grade in test_grades:
    result = system.evaluate({'grade': grade})
    performance = result['performance']

    if performance < 4:
        category = "Needs Improvement"
    elif performance < 7:
        category = "Acceptable"
    else:
        category = "Excellent"

    print(f"{grade:<10.1f} {performance:<15.2f} {category}")

print("=" * 80)

In [ ]:
grades = np.linspace(0, 10, 100)
performances = [system.evaluate({'grade': g})['performance'] for g in grades]

plt.figure(figsize=(12, 6))
plt.plot(grades, performances, 'g-', linewidth=3, label='Sugeno System Response')
plt.scatter(grades[::10], performances[::10], color='red', s=50, zorder=5)

plt.title('System Response Curve: Grade → Performance', fontsize=14, fontweight='bold')
plt.xlabel('Grade (0-10)', fontsize=12)
plt.ylabel('Performance Output', fontsize=12)
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.xlim([0, 10])
plt.ylim([0, 10])
plt.show()